   ### Introduction
    The goal of the assignment is to implement a prototypical CBIR system. We recommend the use of the CalTech 101 image database http://www.vision.caltech.edu/Image_Datasets/Caltech101/. We recommend that you (for a start) select a subset of say 4-5 categories. When you have checked that everything works you may extend to say 20 categories. For each category, the set of images should be split in two: A training set and a test set (of equal size). The test set must not include images in the training set. When using few categories you may also limit the number of training images (to say 10) per category. Depending on your amount of computational power, for more categories, you may increase the number of training images to the double or more. You should extract visual words using SIFT descriptors (ignoring position, orientation and scale) or similar descriptors extracted at interest points. To compute the descriptors, we recommend to use OpenCV's sift, but other options are possible.

### Codebook Generation
    In order to generate a code book, select a set of training images. Then Extract SIFT features from the training images (ignore position, orientation and scale). The SIFT features should be concatenated into a matrix, one descriptor per row. Then you should run the k-means clustering algorithm on the subset of training descriptors to extract good prototype (visual word) clusters. A reasonable k should be small (say between 200 and 500) for a small number of categories (say 5) and larger (say between 500 and 2000) for a larger number of categories. Also, a good value of k may depend on the complexity of your data. You should experiment with a few di erent values of k (but beware that this can be rather time-consuming). Once clustering has been obtained, classify each training descriptor to the closest cluster centers) and form the bag of words (BoW) for each image in the image training set. Note that there may exist several implementations of k-means available in several libraries, e.g. in OpenCV and in scikit-image. These implementations may dffer both with respect to function, parameters and processing time.

  ### Indexing
    The next step consists in content indexing. For each image in the test set you
    should:
       
    a) Extract the SIFT descriptors of the feature points in the image
    b) Project the descriptors onto the codebook, i.e., for each descriptor the closest cluster prototype should be found
    c) Construct the generated corresponding bag of words, i.e, word histogram.

    Please note that you have already performed the same steps for the training images during codebook generation. Now construct and save a table that would contain, per entry at least the file name, the true category, if it belongs to the training- or test set, and the corresponding bag of words / word histogram. The table need only be computed once and then used repeatably in the following retrieval experiments.

### Retrieving
    Finally, you should implement retrieving of images using some of the similarity
    measures discussed in the course slides. You may use:
    
    a) common words
    b) tf-ifd similarity
    c) Bhattacharyya distance or Kullback-Leibler divergence
      
    Please argue for your choice or report the differences in result when applying the different measures. Your report should show commented results for two experiments. In the first you consider retrieving training images. In the second you test how well you can classify test images. Otherwise the two test are identical. For each test you should count:
    
    a) The mean reciprocal rank (i.e. the average across all queries of 1=ranki, where ranki is the rank position of the first correct category for the i'th query).
    b) How often (in per cent) the correct category is in top-3

    Please note that the measures above are just two among a long list of possible performance measures. If you google Information retrieval you may  find alternative measures.

    The report should be kept within 8 pages including everything. About half may show results in form of tables, graphs, and images. Remember to write what the tables and graphs should show. Important decisions, choices, and results should be discussed and explained. In particular, strage/false results should be identified and possible causes should be explained.

In [236]:
# Important packages
import os

import numpy as np
import random

import cv2
import skimage
from skimage.io import imread

import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

In [237]:
# Loading 5 categories from the CALTECH101 corpus
def load_images(main_folder, n_categories):
    '''
    Loads images from multiple categories into one array.
    
    Arguments
    main_folder: folder where subfolders with categories exist
    n_categories: number of categories loaded
    
    Returns
    final_array: array with dimension (n_categories,) consisting of subarrays with images from each category
    cat: order of categories
    '''
    dr = os.listdir(main_folder)
    cat = dr[0:n_categories]
    all_images = []
    all_filenames = []

    for idx_cat, i in enumerate(cat):    
        tmp_path = os.path.join("101_ObjectCategories", i)
        tmp_images = np.array([imread(os.path.join(tmp_path, fname)) for fname in os.listdir(tmp_path) if fname.endswith('.jpg')])
        all_images.append(tmp_images)
        all_filenames.append(os.listdir(tmp_path))
        
    final_array = np.array(all_images)
    return final_array, cat, all_filenames
        
all_images, order, all_filenames = load_images("101_ObjectCategories", 5)        

<ipython-input-237-2d898c49b6a6>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tmp_images = np.array([imread(os.path.join(tmp_path, fname)) for fname in os.listdir(tmp_path) if fname.endswith('.jpg')])
<ipython-input-237-2d898c49b6a6>:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  final_array = np.array(all_images)


In [238]:
all_filenames

[['image_0032.jpg',
  'image_0026.jpg',
  'image_0027.jpg',
  'image_0033.jpg',
  'image_0019.jpg',
  'image_0025.jpg',
  'image_0031.jpg',
  'image_0030.jpg',
  'image_0024.jpg',
  'image_0018.jpg',
  'image_0020.jpg',
  'image_0034.jpg',
  'image_0008.jpg',
  'image_0009.jpg',
  'image_0021.jpg',
  'image_0023.jpg',
  'image_0022.jpg',
  'image_0013.jpg',
  'image_0007.jpg',
  'image_0006.jpg',
  'image_0012.jpg',
  'image_0004.jpg',
  'image_0010.jpg',
  'image_0011.jpg',
  'image_0005.jpg',
  'image_0001.jpg',
  'image_0015.jpg',
  'image_0029.jpg',
  'image_0028.jpg',
  'image_0014.jpg',
  'image_0016.jpg',
  'image_0002.jpg',
  'image_0003.jpg',
  'image_0017.jpg'],
 ['image_0032.jpg',
  'image_0026.jpg',
  'image_0027.jpg',
  'image_0033.jpg',
  'image_0019.jpg',
  'image_0025.jpg',
  'image_0031.jpg',
  'image_0030.jpg',
  'image_0024.jpg',
  'image_0018.jpg',
  'image_0020.jpg',
  'image_0034.jpg',
  'image_0008.jpg',
  'image_0009.jpg',
  'image_0035.jpg',
  'image_0021.jpg',

In [239]:
# Balancing dataset and dividing into test and train 

def train_test_split(image_set, trainsize = 10, testsize = 5):

    # gerenuk = all_images[0]

    all_elements = np.random.choice(len(image_set), trainsize+testsize)

    train_elements = all_elements[1:trainsize+1]
    test_elements = all_elements[-testsize:]

    train_images = image_set[train_elements]
    
    test_images = image_set[test_elements]
    
    return train_images, test_images

In [241]:
# gerenuk = all_images[0]

# Train test split
train = []
train_files = []
test = []
test_files = []

# Applying test_train split to every category
for image_set in all_images:
    train_images, test_images = train_test_split(image_set)
    train.append(train_images)
    test.append(test_images)
    
train = np.array(train)

test = np.array(test)


In [242]:
# Copied from https://docs.opencv.org/3.4/da/df5/tutorial_py_sift_intro.html

test_img = train[0][0]

# Grayscale image
gray = cv2.cvtColor(test_img.copy(), cv2.COLOR_BGR2GRAY)

# Sift engine
sift = cv2.SIFT_create()

# Key points
kp = sift.detect(gray, None)

# Image with keypoints
img = cv2.drawKeypoints(gray, kp, test_img.copy(), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
kp, des = sift.compute(gray,kp)

In [243]:
def sift_descriptors(img):
    if len(img.shape) > 2:
            # Grayscale image
        gray = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()
        # Sift engine
    sift = cv2.SIFT_create()
    # Key points
    kp = sift.detect(gray, None)
    # Image with keypoints
    # img = cv2.drawKeypoints(gray, kp, test_img.copy(), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    _ , des = sift.compute(gray, kp)

    return des


# The SIFT features should be concatenated into a matrix, one descriptor per row.
for set_idx, train_sets in enumerate(train):    
    for img in train_sets:
        des = sift_descriptors(img)        
        if set_idx == 0:
            descriptors = des
        else:
            #np.append(descriptors, des, axis=0)
            np.concatenate((descriptors, des), axis=0)
            #np.vstack((descriptors, des))

In [244]:
# Then you should run the k-means clustering algorithm on the subset of training descriptors to extract good prototype (visual word) clusters. 
# A reasonable k should be small (say b$etween 200 and 500) for a small number of categories (say 5) 
# and larger (say between 500 and 2000) for a larger number of categories. 
# Also, a good value of k may depend on the complexity of your data.

In [245]:
# Inspiration from https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
# Maybe use this link to demonstrate why scipy is nice https://hdbscan.readthedocs.io/en/latest/performance_and_scalability.html#comparison-of-high-performance-implementations
from scipy.cluster.vq import vq, kmeans, whiten

# Normalize (by standard deviation)
whitened = whiten(descriptors)
codebook, distortion = kmeans(whitened, 200)


# Assigns a code from a code book to each observation. 
# Each observation vector in the ‘M’ by ‘N’ obs array is compared with the centroids in the code book and assigned the code of the closest centroid.
# The code book is usually generated using the k-means algorithm. 
# Each row of the array holds a different code, and the columns are the features of the code.
# assigned_features = vq(whitened, codebook)

# whitened[:, 0], whitened[:, 1]
# is
# codebook[:, 0], codebook[:, 1]

In [246]:
print(whitened.shape)
print(descriptors.shape)
# Nice description: https://www.unioviedo.es/compnum/labs/new/kmeans.html

(730, 128)
(730, 128)


In [247]:
# vq returns:
# code - A length M array holding the code book index for each observation.
# dist - The distortion (distance) between the observation and its nearest code.
code_idx, dist = vq(whitened, codebook)

In [249]:
print(code_idx[0]) # Descriptor number 0 has prototype at codebook index 163 
codebook[code_idx[0]] # And that prototype looks like this 

163


array([0.8663638 , 0.771305  , 0.69173235, 0.54447997, 0.15860964,
       0.04201819, 0.16977328, 0.32766712, 0.77998805, 1.7848425 ,
       1.7181492 , 0.35854688, 0.04632722, 0.03957662, 0.01229949,
       0.20783515, 0.11275943, 1.0003595 , 4.1879134 , 3.1920516 ,
       0.19057961, 0.04719809, 0.03523283, 0.08143103, 0.02259532,
       0.2909345 , 3.0537775 , 3.6235478 , 0.19690686, 0.03522928,
       0.10290167, 0.20894162, 0.55486125, 0.1894089 , 0.23230074,
       1.0746602 , 0.43595812, 0.1487209 , 0.22226147, 0.219624  ,
       2.6009781 , 1.1847336 , 0.47451186, 0.23045476, 0.26166275,
       0.43751708, 0.6072514 , 2.1846387 , 0.5165382 , 0.5113899 ,
       1.8540624 , 2.3846767 , 3.1372316 , 1.4169523 , 0.6361868 ,
       1.0997913 , 1.6547092 , 0.9151357 , 1.0036927 , 1.1578718 ,
       0.558645  , 0.28171107, 0.8289141 , 2.1941402 , 0.320693  ,
       0.15846561, 0.1516094 , 0.28293324, 0.26785526, 0.6249685 ,
       0.6096483 , 0.32959163, 2.1943471 , 1.895902  , 1.36353

In [250]:
# Indexing
# Now construct and save a table that would contain, per entry at least the file name, the true category, if it belongs to the training- or test set, and the corresponding bag of words / word histogram. 
# The table need only be computed once and then used repeatably in the following retrieval experiments.

from collections import Counter
bow_train = []

# The SIFT features should be concatenated into a matrix, one descriptor per row.
for set_idx, train_sets in enumerate(train): 
    print(order[set_idx])
    for img in train_sets:
        # Find descriptors
        des = sift_descriptors(img)
        # Whiten
        single_whit = whiten(des)
        # Find distribution of codes
        single_code_idx, _ = vq(single_whit, codebook)
        # Histogram
        #c = Counter(single_code_idx)
        imhist = np.zeros((codebook.shape[0]))
        for idx in single_code_idx:
            imhist[idx] += 1
        # filename
        l = ["train", order[set_idx], imhist]
    
        bow_train.append(l)

bow_test = []       
for set_idx, test_sets in enumerate(test): 
    for img in test_sets:
        # Find descriptors
        des = sift_descriptors(img)
        # Whiten
        single_whit = whiten(des)
        # Find distribution of codes
        single_code_idx, _ = vq(single_whit, codebook)
        # Histogram
        imhist = np.zeros((codebook.shape[0]))
        for idx in single_code_idx:
            imhist[idx] += 1
        #c = Counter(single_code_idx)
        
        l = ["test", order[set_idx], imhist]
        
        bow_test.append(l)

           

gerenuk
hawksbill
headphone
ant
butterfly


In [251]:
# Now we have a list with histograms of prototypes per train image 
bow_train = np.array(bow_train)

# Now we have a list with histograms of prototypes per test image 
bow_test = np.array(bow_test)

<ipython-input-251-07f4e3dea0bf>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bow_train = np.array(bow_train)
<ipython-input-251-07f4e3dea0bf>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bow_test = np.array(bow_test)


In [252]:
from scipy.special import kl_div 
score = []
kls = []
# Distances 
test_hist = bow_test[0]

for hist in bow_test:
    # compute the distance between the two histograms
    # using the method and update the results dictionary
    # kls.append(kl_div(test_hist[2], hist[2]))
    # Formula taken from https://github.com/JaggerWu/images-retrieving/blob/master/CodeBook_generation.py
    score.append(np.sqrt(sum((test_hist[2]-hist[2])**2)))

In [253]:
score = np.array(score)

In [255]:
min_location = np.where(score == min(score))

In [256]:
score[min_location]

array([0.])